In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import matplotlib.pyplot as plt

# 1. Load pre-trained model
model = MobileNet(weights='imagenet')

# 2. Load and preprocess image
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = preprocess_input(x[np.newaxis, ...])  # Add batch dimension

# 3. Get target class (top prediction)
preds = model.predict(x)
target_class = np.argmax(preds[0])
print("Predicted class:", target_class)

# 4. Create CAM model (outputs last conv layer + predictions)
last_conv_layer = model.get_layer('conv_pw_13_relu')  # MobileNet's final conv layer
cam_model = tf.keras.Model(
    inputs=model.input,
    outputs=[last_conv_layer.output, model.output]
)

# 5. Get activations and class weights
with tf.GradientTape() as tape:
    conv_outputs, predictions = cam_model(x)
    loss = predictions[:, target_class]

# 6. Compute weights (GAP equivalent)
weights = tf.reduce_mean(conv_outputs, axis=(1, 2))  # Global Average Pooling

# 7. Generate CAM heatmap
cam = tf.reduce_sum(weights * conv_outputs, axis=-1)
cam = cam[0]  # Remove batch dim

# 8. Post-process CAM
cam = np.maximum(cam, 0)  # ReLU
cam /= np.max(cam)  # Normalize

# 9. Resize to original image
cam = tf.image.resize(cam[..., np.newaxis], (224, 224)).numpy().squeeze()

# 10. Visualize
plt.imshow(img)
plt.imshow(cam, cmap='jet', alpha=0.5)  # Overlay heatmap
plt.title(f'CAM for class {target_class}')
plt.axis('off')
plt.show()